<a href="https://colab.research.google.com/github/shreyamh795/chatbox/blob/main/Copy_of_train_textbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing necessary libraries
!pip install pyMuPDF
import urllib.request
import fitz
import re
import numpy as np
import tensorflow_hub as hub

import os
from sklearn.neighbors import NearestNeighbors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 18.7 MB/s eta 0:00:00


In [2]:
# defining the required functions
def download_pdf(url, output_path):
    urllib.request.urlretrieve(url, output_path)


def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text


def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list


def text_to_chunks(texts, word_length=150, start_page=1):
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []

    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[{idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [3]:
# loading the textbook
download_pdf('https://books.ebalbharati.in/pdfs/703020012.pdf','book.pdf')
#converting into .txt format
pages=pdf_to_text('book.pdf')
#creating chunks from the text generated
chunks=text_to_chunks(pages)

In [5]:
total_words=0
for page in pages:
  total_words +=len(page.split(' '))

print("the toalwords are ", total_words)
print("length of chunks is",len(chunks))

the toalwords are  50906
length of chunks is 340


In [6]:
#displaying random number of chunks
import random
random.seed(123)
random_idxs = random.choices(range(0, len(chunks)), k=5)

for idx in random_idxs:
  print(chunks[idx])
  print()

[12] "Tie plastic bags loosely around the leaves of these plants and keep them in sunlight from early in the morning. In the afternoon, bring those pots into the classroom and observe them. Has the same quantity of water collected in both bags ? Desert plants are either leafless or their leaves are like small needles or have been modified into thorns. As a result, they lose very little water by evaporation. The stem stores water and food and is therefore fleshy. The stems are green as they perform photosynthesis in the absence of leaves. Their roots penetrate deep into the soil and some roots spread away into the soil in search of water. There is a thick layer of a waxy substance on the stems of these plants, too. 1.4 Cactus are thin and slender like a ribbon. This shape helps them to withstand fast currents of water. Air spaces"

[18] "– the first part is ‘genus’ and second, ‘species’. All identified organisms have been assigned a binomial name as per the guidelines of the Internation

In [8]:
#defining the class for creation of vector space
class SemanticSearch:

    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False


    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True


    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]

        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors


    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [10]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [11]:
recommender = SemanticSearch()
recommender.fit(chunks)

In [16]:
recommender('what is the living world')

['[42] "animals like dogs and buffalloes ? Which are those small animals ? 2. From where do these little animals obtain 4.11 Parasites their food ? 3. From where do the worms in the intestine obtain their food ? Some animals depend upon other animals for food. They can obtain the food only from animals on whom they are dependent. This is called as parasitic nutrition. Some animals live on the body surface of other animals and obtain their food by sucking their blood. This is called ectoparasitic nutrition and such animals are called ectoparasites, for example, louse, bed-bug, tick, etc. Animals like tapeworm and roundworms live inside the body of other animals and obtain their food. This is endoparasitic nutrition and these animals are known as endoparasites. The anteater is found in Central and South America. The bobcat or udmanjar is found in India. Obtain its pictures from the internet."',
 '[114] "causes the hardness of water ? Exercise Find out.  104 104 1. What is meant by natura